In [1]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
#import pandas as pd
import numpy as np
from transformers import BartTokenizer, BartForCausalLM, BartForConditionalGeneration, BeamSearchScorer, LogitsProcessorList, MinLengthLogitsProcessor, TopKLogitsWarper, TemperatureLogitsWarper, BartModel
import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import BartTokenizer, BartForCausalLM, BartForConditionalGeneration, BeamSearchScorer, LogitsProcessorList, MinLengthLogitsProcessor, TopKLogitsWarper, TemperatureLogitsWarper
import torch

import math
import random
import re
import argparse


def encode_sentences(tokenizer, df, source_keys, targets, max_length=1024, pad_to_max_length=True, return_tensors="pt"):
    def run_bart(snippet):
        encoded_dict = tokenizer(
          snippet,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
        )
        return encoded_dict

    def get_encoding(snippet, key):
        #print(snippet)
        if isinstance(snippet, list):
            snippet_processed = []
            for each in snippet:
                enc = run_bart(each)
                if len(enc['input_ids']) < 1000:
                    ##key = "attribute"
                    each = "<%s> "%key + each+" </%s>"%key
                    snippet_processed.append(each)
            snippet = " ".join(snippet_processed)
        #print(snippet)
        encoded_dict = run_bart(snippet)
        return encoded_dict

    encoded_sentences = {}

    target_ids = []

    for key in source_keys:
        id_key = '%s_ids'%key
        attention_mask_key = '%s_attention_masks'%key
        if key not in encoded_sentences:
            encoded_sentences[id_key] = []
            encoded_sentences[attention_mask_key] = []
        df_val = list(df[key].values)

        for sentence in df_val:
            sentence_encoding = get_encoding(sentence, key)
            encoded_sentences[id_key].append(sentence_encoding['input_ids'])
            encoded_sentences[attention_mask_key].append(sentence_encoding['attention_mask'])
    
    for tgt_sentence in targets:
        encoded_dict = tokenizer(
              tgt_sentence,
              max_length=max_length,
              padding="max_length" if pad_to_max_length else None,
              truncation=True,
              return_tensors=return_tensors,
              add_prefix_space = True
        )
        # Shift the target ids to the right
        #shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
        target_ids.append(encoded_dict['input_ids'])
    
    for key in list(encoded_sentences.keys()):
        encoded_sentences[key] = torch.cat(encoded_sentences[key], dim = 0)
        
    target_ids = torch.cat(target_ids, dim = 0)
    encoded_sentences['labels'] = target_ids

    return encoded_sentences

In [2]:
val_file = '/Users/sanjana/summarization/datasets/dev_rr_data.csv'
val_df = pd.read_csv(val_file)
val_df

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sanjana/summarization/datasets/robotreviewer/dev_rr_data.csv'